<a href="https://colab.research.google.com/github/saoleksii/Trump2020_CSS_course_project/blob/main/Trump2020_CSS_course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect google drive

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
import zipfile
import os
import time

Unzip dataset

In [36]:
#!unzip "/content/drive/MyDrive/Trump2020_CSS_course_project/political_dataset.zip" -d "/content/drive/MyDrive/Trump2020_CSS_course_project/"

File path

In [37]:
file_path = '/content/drive/MyDrive/Trump2020_CSS_course_project/unprocessed_political_dataset.csv'

# Statistics for not prepared dataset

We use chunks to process 6gb file

In [39]:

num_rows = 0
total_submissions = 0
total_comments = 0
frequency_by_subreddit = {}
size = os.path.getsize(file_path) / (1024**2)
i = 0
df = pd.read_csv(file_path, chunksize=1000000, low_memory=False)
for idx, chunk in enumerate(df):
    i+=1
    if idx == 0:
        print("\nDataframe overview")
        display(chunk.head(10))
        print("\n")
    num_rows+=len(chunk)
    chunk[['title', 'body']] = chunk[['title', 'body']].fillna('')
    submissions_in_chunk = chunk[chunk['title'] != ''].shape[0]
    comments_in_chunk = chunk[chunk['body'] != ''].shape[0]
    total_submissions += submissions_in_chunk
    total_comments += comments_in_chunk
    current_frequency = chunk['subreddit'].value_counts().to_dict()
    for sub, count in current_frequency.items():
        frequency_by_subreddit[sub] = frequency_by_subreddit.get(sub, 0) + count
    print(f'\r{i} of 38 chunks processed', end='', flush=True)
print("\nCompleted\n")

metrics_data = {
    'Total Rows': f"{num_rows}",
    'Total Size (MB)': f"{size}",
    'Submissions Count': f"{total_submissions}",
    'Comments Count': f"{total_comments}",
}
print("Summary of Basic Dataset Metrics:")
display(pd.Series(metrics_data))
print("\nSubreddit_frequency:")
display(pd.Series(frequency_by_subreddit).sort_values(ascending = False))


Dataframe overview


,subreddit,created_utc,title,selftext,body
0,Conservative,1.609459e+09,NaN,NaN,The Boer will rise again
1,Conservative,1.609459e+09,NaN,NaN,Tired of reporting this thread? Debate us on d...
2,Conservative,1.609459e+09,NaN,NaN,[removed]
3,Conservative,1.609459e+09,NaN,NaN,[removed]
4,Conservative,1.609459e+09,NaN,NaN,[removed]
5,Conservative,1.609459e+09,NaN,NaN,Lol. U wish that much was going to stimulus c...
6,Conservative,1.609459e+09,NaN,NaN,[removed]
7,Conservative,1.609459e+09,NaN,NaN,[removed]
8,Conservative,1.609459e+09,NaN,NaN,[removed]
9,Conservative,1.609459e+09,NaN,NaN,[removed]




38 of 38 chunks processed
Completed

Summary of Basic Dataset Metrics:


,0
Total Rows,37433549
Total Size (MB),6886.546831130981
Submissions Count,1139662
Comments Count,36293503



Subreddit_frequency:


,0
politics,19523863
news,5681475
worldnews,4652898
conspiracy,3656420
Conservative,3357061
PoliticalDiscussion,340274
democrats,158427
republicans,38807
NeutralPolitics,24323
t1_gf913op,1


# Preparing dataset:

Check the data by looking at one chunk

In [40]:
df = pd.read_csv(file_path, chunksize=500000)
chunk = next(df) #first chunk
display(chunk.head(10))
chunk[['title', 'selftext', 'body']] = chunk[['title', 'selftext', 'body']].fillna('') # замінити всі NaN на пустий рядок
display(chunk[['title', 'selftext', 'body']].value_counts().head(50))

,subreddit,created_utc,title,selftext,body
0,Conservative,1.609459e+09,NaN,NaN,The Boer will rise again
1,Conservative,1.609459e+09,NaN,NaN,Tired of reporting this thread? Debate us on d...
2,Conservative,1.609459e+09,NaN,NaN,[removed]
3,Conservative,1.609459e+09,NaN,NaN,[removed]
4,Conservative,1.609459e+09,NaN,NaN,[removed]
5,Conservative,1.609459e+09,NaN,NaN,Lol. U wish that much was going to stimulus c...
6,Conservative,1.609459e+09,NaN,NaN,[removed]
7,Conservative,1.609459e+09,NaN,NaN,[removed]
8,Conservative,1.609459e+09,NaN,NaN,[removed]
9,Conservative,1.609459e+09,NaN,NaN,[removed]


title  selftext  body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
                 [removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      175702
                 [deleted]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       21407
                 Tired of reporting this thread? Debate us on discord instead: https://discord.gg/conservative - This is an automated message that appears when probable report abuse is detected. We've found this can lead to a productive discussion in an environment better suited for that sort of thing.\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/Conservative) if you have any questions or concerns.*      1088
                 Looking for debate? Head to the public sections of our discord instead. https://discord.gg/conservative\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/Conservative) if you have any questions or concerns.*                                                                                                                                                                                              950
                 Lol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               176
                 Good                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              140
                 Good.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

Delete top 50 frequent comments

In [41]:
chunk['created_utc'] = pd.to_datetime(chunk['created_utc'], unit='s') #edit time
#delete useless comments
top_50_noise = chunk['body'].value_counts().head(50).index.tolist()
rows_to_drop = chunk['body'].isin(top_50_noise)
indices_to_drop = chunk.index[rows_to_drop]
chunk.drop(indices_to_drop, inplace=True)
display(chunk['body'].value_counts().head(5))

,count
body,
Amen!,20
Thank you.,20
Ok,20
Nah,19
Thanks!,19


Check the size of chunk after removing useless comments

In [42]:
print(chunk.shape)

(298777, 5)


We reduced the number of rows from 500000 to 300000.
Now make one column for title, selftext and body

In [43]:
chunk['text'] = (chunk['title'] + ' ' + chunk['selftext'] + ' ' + chunk['body'])
chunk['text'] = chunk['text'].str.lower().str.strip()
chunk.drop(columns=['title', 'selftext', 'body'], inplace=True)
display(chunk.sample(10))

,subreddit,created_utc,text
470387,Conservative,2021-01-25 14:37:56,there's that misinformation i was talking about
433652,Conservative,2021-01-23 17:05:22,"no shit, ol puddin pants is going balls to the..."
115277,Conservative,2021-01-08 18:38:20,more likely someone figured out how to black h...
425689,Conservative,2021-01-23 06:35:31,i am beginning to feel as though biden being p...
344059,Conservative,2021-01-20 21:25:22,he never got over being outed as a druggie
386111,Conservative,2021-01-21 22:21:58,you're retarded. i genuinely mean that.
101616,Conservative,2021-01-08 01:03:55,it’s about time
424440,Conservative,2021-01-23 05:04:21,joe biden was just walking around without a mask.
402540,Conservative,2021-01-22 14:16:06,"no longer ""*the toughest gun control laws in t..."
341721,Conservative,2021-01-20 20:40:08,or the rolling stones on 1/12/15 for their 50t...


If text is less than 7 symbols it is useless so we delete everything that is less than it

In [44]:
drop_rows = chunk['text'].str.len() <= 7
indices_to_drop = chunk.index[drop_rows]
chunk.drop(indices_to_drop, inplace = True)
display(chunk.sample(10))
print(chunk.shape)

,subreddit,created_utc,text
232872,Conservative,2021-01-14 17:54:54,"as long as you walk blindly towards all, then ..."
453794,Conservative,2021-01-24 17:39:35,guess who’s the new joseph goebbels 2.0 ?\n\nh...
389520,Conservative,2021-01-22 00:24:50,"most scientists don’t do bench work. many do, ..."
364620,Conservative,2021-01-21 10:04:07,"y’all are truly a sad, deplorable bunch. did y..."
439101,Conservative,2021-01-23 22:02:46,&gt;the guy barely slept and worked nonstop\n\...
267767,Conservative,2021-01-17 21:40:47,"i think we’re pretty far past that, at this po..."
303963,Conservative,2021-01-19 21:32:56,0 bombs used under his administration. he said...
152590,Conservative,2021-01-10 12:11:46,maybe the left need to be reminded or informed...
374584,Conservative,2021-01-21 16:31:17,how much money do these workers deserve in you...
388747,Conservative,2021-01-21 23:57:08,can’t hide as an empty suit now joe.


(295351, 3)


Now select key words to keep only rows that are needed

In [45]:
keyword_list = [
    'trump', 'biden', 'pence', 'capitol', 'capitol hill', 'impeach', 'impeachment',
    'january 6', '6th jan', 'election', 'stolen', 'steal', 'fraud', 'stop the steal',
    'deep state', 'conspiracy', 'patriot', 'traitors', 'treason', 'rigged',
    'censor', 'ban', 'removed', 'suspended', 'free speech', 'freedom of speech',
    'twitter ban', 'parler', 'maga'
]
keywords = '|'.join(keyword_list)
rows_to_drop = ~chunk['text'].str.contains(keywords)
indices_to_drop = chunk.index[rows_to_drop]
new_chunk = chunk.drop(indices_to_drop)
display(new_chunk.head(10))
print(new_chunk.shape)

,subreddit,created_utc,text
21,Conservative,2021-01-01 00:03:03,i wonder how trump feels about pence right now?
23,Conservative,2021-01-01 00:03:34,this man got his upvotes silenced and then got...
32,Conservative,2021-01-01 00:04:34,of course this is what the demoncrats want. a ...
46,Conservative,2021-01-01 00:07:13,a bank is a private business and should be abl...
51,Conservative,2021-01-01 00:08:50,or tried for treason/sedition?
56,Conservative,2021-01-01 00:09:26,"let's be real, the optics of pence literally p..."
61,Conservative,2021-01-01 00:10:29,i have a feeling trump knows. both trump and p...
76,Conservative,2021-01-01 00:12:40,&gt;\ti honestly think he’s purposely trying t...
84,Conservative,2021-01-01 00:14:09,when has trump supporters rioted or caused vio...
94,Conservative,2021-01-01 00:15:37,its called irony. they let the last one be sto...


(75510, 3)


Apply for all chunks

In [46]:
output_file = '/content/drive/MyDrive/Trump2020_CSS_course_project/processed_political_dataset.csv'
num_rows = 0
frequency_by_subreddit = {}
i = 0
df = pd.read_csv(file_path, chunksize=1000000, low_memory=False)
for idx, chunk in enumerate(df):
  chunk[['title', 'selftext', 'body']] = chunk[['title', 'selftext', 'body']].fillna('') # replace NaN with empty line
  chunk.rename(columns={'created_utc': 'time_created'}, inplace=True)
  chunk['time_created'] = pd.to_datetime(chunk['time_created'], unit='s') #edit time
  #delete useless comments
  top_50_noise = chunk['body'].value_counts().head(50).index.tolist()
  rows_to_drop = chunk['body'].isin(top_50_noise)
  indices_to_drop = chunk.index[rows_to_drop]
  chunk.drop(indices_to_drop, inplace=True)
  #merge text columns
  chunk['text'] = (chunk['title'] + ' ' + chunk['selftext'] + ' ' + chunk['body'])
  chunk['text'] = chunk['text'].str.lower().str.strip()
  chunk.drop(columns=['title', 'selftext', 'body'], inplace=True)
  #delete less than 7 symbols
  drop_rows = chunk['text'].str.len() <= 7
  indices_to_drop = chunk.index[drop_rows]
  chunk.drop(indices_to_drop, inplace = True)
  #select keywords
  keyword_list = [
    'trump', 'biden', 'pence', 'capitol', 'capitol hill', 'impeach', 'impeachment',
    'january 6', '6th jan', 'election', 'stolen', 'steal', 'fraud', 'stop the steal',
    'deep state', 'conspiracy', 'patriot', 'traitors', 'treason', 'rigged',
    'censor', 'ban', 'removed', 'suspended', 'free speech', 'freedom of speech',
    'twitter ban', 'parler', 'maga'
    ]
  keywords = '|'.join(keyword_list)
  rows_to_drop = ~chunk['text'].str.contains(keywords)
  indices_to_drop = chunk.index[rows_to_drop]
  chunk.drop(indices_to_drop, inplace=True)

  #statistics
  i+=1
  if idx == 0:
    print("\nDataframe overview")
    display(chunk.head(10))
    print(chunk.shape)
    print("\n")
  num_rows+=len(chunk)
  current_frequency = chunk['subreddit'].value_counts().to_dict()
  for sub, count in current_frequency.items():
      frequency_by_subreddit[sub] = frequency_by_subreddit.get(sub, 0) + count

  #processed csv file ---------- uncomment to save
  # if idx == 0:
  #     header_flag = True
  #     mode_flag = 'w'
  # else:
  #     header_flag = False
  #     mode_flag = 'a'
  # chunk.to_csv(output_file, index=False, mode=mode_flag, header=header_flag, encoding='utf-8')
  print(f'\r{i} of 38 chunks processed', end='', flush=True)
print("\nCompleted\n")

size = os.path.getsize(output_file) / (1024**2)
metrics_data = {
    'Total Rows': f"{num_rows}",
    'Total Size (MB)': f"{size}",
}
print("Summary of Basic Dataset Metrics:")
display(pd.Series(metrics_data))
print("\nSubreddit_frequency:")
display(pd.Series(frequency_by_subreddit).sort_values(ascending = False))


Dataframe overview


,subreddit,time_created,text
21,Conservative,2021-01-01 00:03:03,i wonder how trump feels about pence right now?
23,Conservative,2021-01-01 00:03:34,this man got his upvotes silenced and then got...
32,Conservative,2021-01-01 00:04:34,of course this is what the demoncrats want. a ...
46,Conservative,2021-01-01 00:07:13,a bank is a private business and should be abl...
51,Conservative,2021-01-01 00:08:50,or tried for treason/sedition?
56,Conservative,2021-01-01 00:09:26,"let's be real, the optics of pence literally p..."
61,Conservative,2021-01-01 00:10:29,i have a feeling trump knows. both trump and p...
76,Conservative,2021-01-01 00:12:40,&gt;\ti honestly think he’s purposely trying t...
84,Conservative,2021-01-01 00:14:09,when has trump supporters rioted or caused vio...
94,Conservative,2021-01-01 00:15:37,its called irony. they let the last one be sto...


(146170, 3)


38 of 38 chunks processed
Completed

Summary of Basic Dataset Metrics:


,0
Total Rows,5917521
Total Size (MB),2055.3090267181396



Subreddit_frequency:


,0
politics,3937771
conspiracy,528389
news,501809
Conservative,470176
worldnews,350372
PoliticalDiscussion,93647
democrats,22183
republicans,7481
NeutralPolitics,5693
